In [1]:
import numpy as np
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib import pyplot as plt
import seaborn as sns
import string
import re
from nltk.stem import WordNetLemmatizer as wnl
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')



import torch.autograd as tag

[nltk_data] Downloading package punkt to C:\Users\Sofia
[nltk_data]     Beyerlein\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Sofia
[nltk_data]     Beyerlein\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Sofia
[nltk_data]     Beyerlein\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Sofia
[nltk_data]     Beyerlein\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
#Download training and testing data
file_path1 = r'C:\Users\Sofia Beyerlein\Desktop\Cornell Graduate\Applied Machine Learning\hw2\test.csv'
test = pd.read_csv(file_path1)
test_ids = test['Id']

file_path2 = r'C:\Users\Sofia Beyerlein\Desktop\Cornell Graduate\Applied Machine Learning\hw2\train.csv'
train = pd.read_csv(file_path2)

In [3]:
#Preprocessing the data, explain preprocessing steps and the reasons why you need to preprocess
#cleaning training
train['LotFrontage'].fillna(train['LotFrontage'].median(), inplace=True)
train['LotArea'].fillna(train['LotArea'].median(), inplace=True)
train['YearBuilt'].fillna(train['LotArea'].median(), inplace=True)
train['LotShape'].fillna('None', inplace=True)
train['BldgType'].fillna('None', inplace=True)
train['HouseStyle'].fillna('None', inplace=True)

#cleaning testing
test['LotFrontage'].fillna(test['LotFrontage'].median(), inplace=True)
test['LotArea'].fillna(test['LotArea'].median(), inplace=True)
test['YearBuilt'].fillna(test['LotArea'].median(), inplace=True)
test['LotShape'].fillna('None', inplace=True)
test['BldgType'].fillna('None', inplace=True)
test['HouseStyle'].fillna('None', inplace=True)


In [4]:
#one-hot encoding
test_combined = pd.get_dummies(test, drop_first = True)
train_combined = pd.get_dummies(train, drop_first= True)

test_combined = test_combined.reindex(columns=train_combined.columns, fill_value=0)

1. Continuation of House Prices from HW1

In [5]:
#PART A

# Using OLS
columns_to_include = ['LotFrontage', 'LotArea', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'BldgType_2fmCon', 'BldgType_Duplex', 'BldgType_Twnhs', 'BldgType_TwnhsE', 'HouseStyle_1.5Unf', 'HouseStyle_1Story', 'HouseStyle_2.5Unf', 'HouseStyle_2Story', 'HouseStyle_SFoyer', 'HouseStyle_SLvl', 'YearBuilt']

target = 'SalePrice'

train_combined = train_combined.drop(columns=['GarageYrBlt'])
train_combined = train_combined.drop(columns=['MasVnrArea'])

#turning the feats into tensors
x_train = np.array(train_combined.drop(columns = [target]), dtype=np.float32)
y_train = np.array(train_combined[target], dtype=np.float32)
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1,1)

print("NaN values in x_train:", torch.isnan(x_train).any().item())
print("NaN values in y_train:", torch.isnan(y_train).any().item())


class LinearRegression(nn.Module):
    def __init__(self, input_dim):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
        
    def forward(self, x):
        return self.linear(x)
    
#model instance
input_dim = x_train.shape[1]
model = LinearRegression(input_dim)

#loss
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

#train model
num_epochs = 1000
for epoch in range(num_epochs):
    outputs = model(x_train)
    loss = criterion(outputs, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
with torch.no_grad():
    y_pred = model(x_train).numpy()

#mse
mse = mean_squared_error(y_train.numpy(), y_pred)
root_mse = np.sqrt(mse)
print(f"MSE: {mse}")
print(f"Root MSE: {root_mse}")

#r^2
r2 = r2_score(y_train.numpy(), y_pred)
print(f"r^2 scoreL {r2}")

NaN values in x_train: False
NaN values in y_train: False
MSE: 2400779520.0
Root MSE: 48997.75
r^2 scoreL 0.6193340572519818


In [6]:
#PART B
#In the HW2 - Report

2. Data generating distribution and convergence of linear regression

In [7]:
#PART A
#made function so it could be used easily in next part

def regression_r2(num_samples):
    #setting up variables
    mean_X = 168
    std_X = 30
    mean_eps = 0
    std_eps = 20
    alpha = 20
    beta = 0.5

    #filling X and epsilon
    X = np.random.normal(mean_X, std_X, num_samples).reshape(-1, 1)
    epsilon = np.random.normal(mean_eps, std_eps, num_samples).reshape(-1, 1)

    #making Y function
    Y = alpha + beta * X + epsilon
    
    #making prediction
    model = LinearRegression()
    model.fit(X, Y)
    y_predict = model.predict(X)

    #calculating r2 
    r2 = r2_score(Y, y_predict)
    
    return r2, model.coef_[0, 0], model.intercept_[0]

In [11]:
#PART B

print("R2 score, coefficient, intercept:")
print(regression_r2(pow(10, 2)))
print(regression_r2(pow(10, 3)))
print(regression_r2(pow(10, 4)))
print(regression_r2(pow(10, 5)))
print(regression_r2(pow(10, 6)))

R2 score, coefficient, intercept:


TypeError: LinearRegression.__init__() missing 1 required positional argument: 'input_dim'

In [ ]:
#PART C

#Written in the Homework 2 - Report

#PART D
#Written in the Homework 2 - Report

#PART E EXTRA CREDIT

#PART F
#Written in the Homework 2 - Report

#PART G
#Written in the Homework 2 - Report

3. Binary Classification on Text Data

In [12]:
#PART A

#Questions answered in Homework 2 - Report
filepath_train = (r"C:\Users\Sofia Beyerlein\Desktop\Cornell Graduate\Applied Machine Learning\hw2\nlp-getting-started\train.csv")
filepath_test = (r'C:\Users\Sofia Beyerlein\Desktop\Cornell Graduate\Applied Machine Learning\hw2\nlp-getting-started\test.csv')

train = pd.read_csv(filepath_train)
test = pd.read_csv(filepath_test)

disaster_count = train[train['target'] == 1].shape[0]
no_disaster_count = train[train['target'] == 0].shape[0]
disaster_percent = (disaster_count/len(train)) * 100
no_disaster_percent = (no_disaster_count/len(train)) * 100
print("disaster percent: ", disaster_percent, "no disaster percent: ", no_disaster_percent)

disaster percent:  42.96597924602653 no disaster percent:  57.03402075397347


In [13]:
#PART B

#70% -> 5329/7613 and 30% -> 2284
training_set = train.sample(frac=0.7)
dev_set = train.drop(training_set.index)

In [14]:
#PART C

def preprocess_data(df):
    words_to_remove = {'the', 'and', 'or'}
    #lowercase
    df['text'] = df['text'].apply(lambda x: x.lower())
    #remove @ and urls
    df['text'] = df['text'].apply(lambda x: re.sub(r'@\S+', '', x))
    #remove # and hashtags
    df['text'] = df['text'].apply(lambda x: re.sub(r'#\S+', '', x))
    #strip punctuation
    df['text'] = df['text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
    #strip the and or
    df['text'] = df['text'].apply(lambda x: ' '.join(word for word in x.split() if word not in words_to_remove))
    #lemmatize
    lemmatizer = nltk.WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    
    def lemmatize_text(text):
        tokens = nltk.word_tokenize(text)
        lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
        return ' '.join(lemmatized_tokens)
    
    df['text'] = df['text'].apply(lemmatize_text)
    
    return df
    
        
preprocess_data(training_set)
preprocess_data(dev_set)

,id,keyword,location,text,target
0,1,NaN,NaN,deed reason may allah forgive u,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,resident asked shelter place notified officer ...,1
4,7,NaN,NaN,got sent photo ruby smoke pours school,1
7,13,NaN,NaN,im top hill see fire wood,1
...,...,...,...,...,...
7591,10846,NaN,NaN,heat wave warning aa ayyo dei plan visit frien...,1
7598,10853,NaN,NaN,fatherofthree lost control car overtaking coll...,1
7600,10855,NaN,NaN,evacuation order lifted town roosevelt httptco...,1
7603,10862,NaN,NaN,official say quarantine place alabama home pos...,1


In [17]:
#PART D

M = 3
vectorizer = CountVectorizer(binary=True, min_df=M)

train_vectors = vectorizer.fit_transform(training_set['text'])
dev_vectors = vectorizer.transform(dev_set['text'])

num_features = len(vectorizer.get_feature_names_out())
print(f"Total number of features: {num_features}")

Total number of features: 3020


In [18]:
#PART E

#part i.
lr_nr = LogisticRegression(penalty=None, max_iter=200)
lr_nr.fit(train_vectors, training_set['target'])
y_train_predict_nr = lr_nr.predict(train_vectors)
y_dev_predict_nr = lr_nr.predict(dev_vectors)
f1_train_nr = f1_score(training_set['target'], y_train_predict_nr)
f1_dev_nr = f1_score(dev_set['target'], y_dev_predict_nr)
print('Logistic Regression without regularization:')
print(f"F1 score training: {f1_train_nr}")
print(f"F1 score dev: {f1_dev_nr}")

#part ii.
lr_l1 = LogisticRegression(penalty='l1', solver = 'liblinear')
lr_l1.fit(train_vectors, training_set['target'])
y_train_predict_l1 = lr_l1.predict(train_vectors)
y_dev_predict_l1 = lr_l1.predict(dev_vectors)
f1_train_l1 = f1_score(training_set['target'], y_train_predict_l1)
f1_dev_l1 = f1_score(dev_set['target'], y_dev_predict_l1)
print('\nLogistic Regression with L1 regularization:')
print(f"F1 score training: {f1_train_l1}")
print(f"F1 score dev: {f1_dev_l1}")

#part iii.
lr_l2 = LogisticRegression(penalty='l2', solver = 'liblinear')
lr_l2.fit(train_vectors, training_set['target'])
y_train_predict_l2 = lr_l2.predict(train_vectors)
y_dev_predict_l2 = lr_l2.predict(dev_vectors)
f1_train_l2 = f1_score(training_set['target'], y_train_predict_l2)
f1_dev_l2 = f1_score(dev_set['target'], y_dev_predict_l2)
print('\nLogistic Regression with L2 regularization:')
print(f"F1 score training: {f1_train_l2}")
print(f"F1 score dev: {f1_dev_l2}")

#part iv.
#This part is written in the report

#part v
feat_names = vectorizer.get_feature_names_out()
coefficients = lr_l1.coef_[0]

positive = coefficients.argsort()[-5:][::-1]
negative = coefficients.argsort()[:5]

print("\nPositive:")
for words in positive:
    print(f"{feat_names[words]} : {coefficients[words]}")
          
print("\nNegative:")
for words in negative:
    print(f"{feat_names[words]} : {coefficients[words]}")         

Logistic Regression without regularization:
F1 score training: 0.9774271312732852
F1 score dev: 0.6596279537456008

Logistic Regression with L1 regularization:
F1 score training: 0.8401210146613916
F1 score dev: 0.706992230854606

Logistic Regression with L2 regularization:
F1 score training: 0.8869604932633022
F1 score dev: 0.7237991266375545

Positive:
typhoon : 3.815250112567738
migrant : 3.5704044448005754
bombing : 3.4775784559068135
drought : 2.9886744754565275
mh370 : 2.9767008480840738

Negative:
dying : -2.191539628516657
bag : -1.971195526669905
armageddon : -1.9593554715689157
love : -1.9474393314500993
anti : -1.8716101580586273


C:\Users\Sofia Beyerlein\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
#PART F

M = 3
vec_2_gram = CountVectorizer(ngram_range=(2, 2), min_df=M)

#fitting the data
x_train_2_gram = vec_2_gram.fit_transform(training_set['text'])
x_dev_2_gram = vec_2_gram.transform(dev_set['text'])

#length of vocab
vocab_len = len(vec_2_gram.get_feature_names_out())
print(f"Num of 2-grams in vocabulary: {vocab_len}")

#printing the pairs of n-gram
rand_2_grams = np.random.choice(vec_2_gram.get_feature_names_out(), 10, replace=False)
print("\nRandom 2-grams in vocab:")
for thing in rand_2_grams:
    print(thing)

#training the models
lr_2_gram = LogisticRegression(penalty='l2')
lr_2_gram.fit(x_train_2_gram, training_set['target'])

#predicting on training and dev set
y_train_pred_2_gram = lr_2_gram.predict(x_train_2_gram)
y_dev_pred_2_gram = lr_2_gram.predict(x_dev_2_gram)

#f1 scores
f1_train_2_gram = f1_score(training_set['target'], y_train_pred_2_gram)
f1_dev_2_gram = f1_score(dev_set['target'], y_dev_pred_2_gram)

print("\nLogistic regression with 2-gram:")
print(f"F1 score training: {f1_train_2_gram}")
print(f"F1 score dev: {f1_dev_2_gram}")

Num of 2-grams in vocabulary: 1504

Random 2-grams in vocab:
added video
terror attack
modified land
freakiest freak
saved many
prime minister
politics grief
20 amp
sandstorm minute
colorado theater

Logistic regression with 2-gram:
F1 score training: 0.6334661354581673
F1 score dev: 0.5505780346820809


In [20]:
#PART G

data_comb = pd.concat([training_set, dev_set], ignore_index=True)

M = 3
vectorizer = CountVectorizer(binary=True, min_df=M)
x_complete = vectorizer.fit_transform(data_comb['text'])

# Train the logistic regression model with L1 regularization
lr_l1 = LogisticRegression(penalty='l1', solver='liblinear')
lr_l1.fit(x_complete, data_comb['target'])

# Predict on the train and test sets
x_test = vectorizer.transform(test['text'])
y_pred = lr_l1.predict(x_test)

final = pd.DataFrame({'id': test['id'], 'target': y_pred})
final.to_csv('submission.csv', index=False)